# Load training data

In [ ]:
%matplotlib inline
%pylab inline

In [6]:
import os
os.chdir('../..')
sys.path.append('../..')
sys.path.insert('I:/menpo/project_lv')
import menpo.io as mio
from menpo.landmark import labeller, left_ventricle_34,left_ventricle_34_trimesh,left_ventricle_34_trimesh1
from menpofit.aam import HolisticAAM,PatchAAM
from menpo.transform import ThinPlateSplines
from menpo.feature import *
from menpofit.aam import LucasKanadeAAMFitter, WibergInverseCompositional
from menpofit.fitter import noisy_shape_from_bounding_box
from menpofit.visualize import plot_cumulative_error_distribution
from menpowidgets import *
from menpodetect import load_dlib_left_ventricle_detector
from menpo.io import export_pickle,import_pickle
from pathlib import Path

# method to load a database
def load_database(path_to_images, crop_percentage,max_diagonal=400, max_images=None):
    images = []
    # load landmarked images
    for i in mio.import_images(path_to_images, max_images=max_images, verbose=True):
           
        # convert it to grayscale if needed
        if i.n_channels == 3:
            i = i.as_greyscale(mode='luminosity')
        # crop image
        i = i.crop_to_landmarks_proportion(crop_percentage)
        
        d = i.diagonal()
        if d > max_diagonal:
            i = i.rescale(float(max_diagonal) / d)
        # define a TriMesh which will be useful for Piecewise Affine Warp of HolisticAAM
        
        labeller(i, 'PTS', left_ventricle_34)#lv_34_trimesh
        labeller(i, 'PTS', left_ventricle_34_trimesh1)
        labeller(i, 'PTS', left_ventricle_34_trimesh)
        
        # append it to the list
        images.append(i)
    return images

Populating the interactive namespace from numpy and matplotlib


C:\Anaconda2\envs\menpodev\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['style', 'diagonal', 'gradient']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
path_to_lfpw = Path('K:/datasets/lv_pts_crop/')
crop_percentage = 0.5
max_images =4240
train_images = load_database(path_to_lfpw/ 'trainset',crop_percentage,max_images=max_images)

#visualize_images(training_images)

Found 4240 assets, index the returned LazyList to import.


# Active Appearance Models

## Training Parameters

In [3]:
from menpo.feature import no_op, fast_dsift

group = 'lv_34'
features = no_op
patch_shape = (17, 17)
diagonal = 100
scales = (1, .5)
max_shape_components = 25
max_appearance_components = 250
normalize_parts = False

## Global PWA

In [7]:
from menpofit.transform import DifferentiablePiecewiseAffine
from alabortijcv2015.aam import GlobalAAMBuilder
from alabortijcv2015.utils import pickle_dump

aam_path = Path('data/deformable/model/no-op-PWA-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else:
    aam = GlobalAAMBuilder(transform=DifferentiablePiecewiseAffine,
                       features=features,
                       diagonal=diagonal,
                       scales=scales,
                       max_shape_components=max_shape_components,
                       max_appearance_components=max_appearance_components).build(training_images,
                                                                                  group=group,verbose=True)
    mio.export_pickle(aam,aam_path, overwrite=True, protocol=2) 
    print 'train Done'

ImportError: No module named alabortijcv2015.aam

## Global TPS

In [ ]:
from menpofit.transform import DifferentiableThinPlateSplines
from alabortijcv2015.aam import GlobalAAMBuilder
 
aam_path = Path('data/deformable/model/no-op-TPS-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else:
    aam = GlobalAAMBuilder(transform=DifferentiableThinPlateSplines,
                       features=features,
                       diagonal=diagonal,
                       scales=scales,
                       max_shape_components=max_shape_components,
                       max_appearance_components=max_appearance_components).build(training_images,
                                                                                  group=group,
                                                                                  verbose=True)
    print 'train Done'   

## Patch

In [ ]:
from alabortijcv2015.aam import PatchAAMBuilder 

aam_path = Path('data/deformable/model/no-op-patch-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else: 
    aam = PatchAAMBuilder(patch_shape=patch_shape,
                      features=features,
                      diagonal=diagonal,
                      scales=scales,
                      max_shape_components=max_shape_components,
                      max_appearance_components=max_appearance_components).build(training_images,
                                                                                 group=group,
                                                                                 verbose=True)
      print 'train Done'

## Linear Global PWA

In [ ]:
from menpofit.transform import DifferentiablePiecewiseAffine
from alabortijcv2015.aam import LinearGlobalAAMBuilder
 
aam_path = Path('data/deformable/model/no-op-PWA-linear-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else:    
    aam = LinearGlobalAAMBuilder(transform=DifferentiablePiecewiseAffine,
                             features=features,
                             diagonal=diagonal,
                             scales=scales,
                             max_shape_components=max_shape_components,
                             max_appearance_components=max_appearance_components).build(training_images,
                                                                                        group=group,
                                                                                        verbose=True)
    print 'train Done'

## Linear Global TPS

In [ ]:
from menpofit.transform import DifferentiableThinPlateSplines
from alabortijcv2015.aam import LinearGlobalAAMBuilder

aam_path = Path('data/deformable/model/no-op-TPS-linear-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else:  
    aam = LinearGlobalAAMBuilder(transform=DifferentiableThinPlateSplines,
                             features=features,
                             diagonal=diagonal,
                             scales=scales,
                             max_shape_components=max_shape_components,
                             max_appearance_components=max_appearance_components).build(training_images,
                                                                                        group=group,
                                                                                        verbose=True)
    print 'train Done'

## Linear Patch

In [ ]:
from alabortijcv2015.aam import LinearPatchAAMBuilder
from alabortijcv2015.utils import pickle_dump

aam_path = Path('data/deformable/model/no-op-patch-linear-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else:     
    aam = LinearPatchAAMBuilder(patch_shape=patch_shape,
                            features=features,
                            diagonal=diagonal,
                            scales=scales,
                            max_shape_components=max_shape_components,
                            max_appearance_components=max_appearance_components).build(training_images,
                                                                                       group=group,
                                                                                       verbose=True)
    print 'train Done'

## Parts

In [ ]:
from alabortijcv2015.aam import PartsAAMBuilder

aam_path = Path('data/deformable/model/no-op-parts-aam.pkl')
if aam_path.exists():
    aam= mio.import_pickle(aam_path)
    print 'load Done'
else: 
    aam = PartsAAMBuilder(parts_shape=patch_shape,
                     features=features,
                     diagonal=diagonal,
                     normalize_parts=normalize_parts,
                     scales=scales,
                     max_shape_components=max_shape_components,
                     max_appearance_components=max_appearance_components).build(training_images,
                                                                                group=group,
                                                                                verbose=True)
    print 'train Done'